In [5]:
import network as net
import pandas as pd
import numpy as np
import pickle
import re
from copy import deepcopy

#reseau=net.ReseauNeural([832,416,208,104,1])
reseau=net.ReseauNeural([832,1048, 500,50,1])
#how can we calculate accuracy on regression models

parametres2: are already trained with the following code and we're tring to get optimal ones.

parametres0: are just initial parametres ,just to not having random ones everytime

In [6]:
#reseau.biais = pickle.load( open( 'biais4.pkl' , 'rb' ))
#reseau.poids = pickle.load( open( 'poids4.pkl' , 'rb' ))

In [7]:
train = pd.read_csv('chessData.csv')
test = pd.read_csv('tactic_evals.csv')

In [8]:
def fen_to_bit_vector(fen):
    parts = re.split(" ", fen)
    piece_placement = re.split("/", parts[0])
    active_color = parts[1]
    castling_rights = parts[2]
    en_passant = parts[3]
    halfmove_clock = int(parts[4])
    fullmove_clock = int(parts[5])

    bit_vector = np.zeros((13, 8, 8), dtype=np.uint8)

    # piece to layer structure taken from reference [1]
    piece_to_layer = {'R': 1,'N': 2,'B': 3,'Q': 4,'K': 5,'P': 6,'p': 7,'k': 8,'q': 9,'b': 10,'n': 11,'r': 12}

    castling = {'K': (7,7),'Q': (7,0),'k': (0,7),'q': (0,0), }

    for r, row in enumerate(piece_placement):
        c = 0
        for piece in row:
            if piece in piece_to_layer:
                bit_vector[piece_to_layer[piece], r, c] = 1
                c += 1
            else:
                c += int(piece)

    if en_passant != '-':
        bit_vector[0, ord(en_passant[0]) - ord('a'), int(en_passant[1]) - 1] = 1

    if castling_rights != '-':
        for char in castling_rights:
            bit_vector[0, castling[char][0], castling[char][1]] = 1

    if active_color == 'w':
        bit_vector[0, 7, 4] = 1
    else:
        bit_vector[0, 0, 4] = 1

    if halfmove_clock > 0:
        c = 7
        while halfmove_clock > 0:
            bit_vector[0, 3, c] = halfmove_clock%2
            halfmove_clock = halfmove_clock // 2
            c -= 1
            if c < 0:
                break

    if fullmove_clock > 0:
        c = 7
        while fullmove_clock > 0:
            bit_vector[0, 4, c] = fullmove_clock%2
            fullmove_clock = fullmove_clock // 2
            c -= 1
            if c < 0:
                break

    return bit_vector

In [9]:
def create_chess_dataset(data_frame):
    fens = np.array([*map(fen_to_bit_vector, data_frame["FEN"])], dtype=np.float32)
    fens=[*map(lambda x: x.reshape((832,1)), fens)]
    evals = np.array([eval_to_int(x) for x in data_frame["Evaluation"]])
    return fens, evals

def eval_to_int(evaluation):
    try:
        res = int(evaluation)
    except ValueError:
        res = 10000 if evaluation[1] == '+' else -10000
    return res/10000

data train: 5 000 000

mini lots: 30 - 20 - 10

epoques: 10

taux d'apprentissage : 10 - 1 - 0.1 - 0.01

fonctions d'activation : arctan - sigmoid - tanh

In [10]:
n_train= 1000000
n_test=10000

In [11]:
train_data = train.iloc[-n_train:]
test_data=test.iloc[:n_test]

FEN_train, EVA_train = create_chess_dataset(train_data)
FEN_test, EVA_test = create_chess_dataset(test_data)

data_train = list(zip(FEN_train, EVA_train))
data_test = list(zip(FEN_test, EVA_test))

In [12]:
data_train[0]

(array([[0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
      

In [13]:
reseau2=deepcopy(reseau)

with this archetecture : [832,416,208,104,1]
the model achieved MSE: 0.10735

In [14]:
L2=reseau2.Descente_gradient( data_train,1,taille_mini_lot=64,eta=0.01,,f=net.arctan,f_prime=net.arctan_prime,donnees_test=data_test)

TypeError: ReseauNeural.Descente_gradient() missing 1 required positional argument: 'check_lr'

6h 4m

In [ ]:
import pickle

pickle.dump( reseau2.poids , open( 'poids4.pkl' , 'wb' ) )
pickle.dump( reseau2.biais , open( 'biais4.pkl' , 'wb' ) )
#reseau.poids = pickle.load( open( '/content/poids1.pkl' , 'rb' ))
#reseau.biais = pickle.load( open( '/content/biais1.pkl' , 'rb' ))



In [ ]:
x=[L2[i][0] for i in range(len(L2))]
y=[L2[i][1][0][0] for i in range(len(L2))]
import matplotlib.pyplot as plt
plt.plot(x,y)
plt.show()


In [ ]:
#11800-2000
reseau.propagation_directe( FEN_test[1002],net.arctan)-EVA_test[1002]